<a href="https://colab.research.google.com/github/Dav00Arm/Segmentation_masks/blob/main/Seamless_Clone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import glob
import random
from imutils import rotate

def center_crop(img, dim):
    """Returns center cropped image
    :param img: image to be center cropped
    :param dim: dimensions (width, height) to be cropped
    :returns: croped image with shape of dim 
    """
    width, height = img.shape[1], img.shape[0]

    crop_width = dim[0] if dim[0]<img.shape[1] else img.shape[1]
    crop_height = dim[1] if dim[1]<img.shape[0] else img.shape[0] 
    mid_x, mid_y = int(width/2), int(height/2)
    cw2, ch2 = int(crop_width/2), int(crop_height/2) 
    crop_img = img[mid_y-ch2:mid_y+ch2, mid_x-cw2:mid_x+cw2]
    return crop_img


def thicker_seg_mask(src, mask):  
    """
    Making segmentation mask thicker to get cleaner output 
    in seamless clone
    :param src: path of source image
    :param mask: path of segmentation mask of source image
    :returns: mask of shape (w, h, 3)
    """

    if cv2.imread(src).shape == (512, 512, 3):
      dim = (200, 200)
    elif cv2.imread(src).shape == (256, 256, 3):
      dim = (60, 60)

    src = cv2.imread(src) 
    mask = cv2.resize(cv2.imread(mask), dim, interpolation = cv2.INTER_AREA)
    
    new = np.zeros((mask.shape[0], mask.shape[1]), dtype='uint8')
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    mask[mask>0] = 255
    
    for i in range(len(mask)):
        for j in range(len(mask)):
            # FIRST ROW
            if i == 0 and (0 < j < len(mask)-1):
                if mask[0][j] == 255:
                    new[0][j-1] = 255
                    new[0][j] = 255
                    new[0][j+1] = 255
                    new[1][j-1] = 255
                    new[1][j] = 255
                    new[1][j+1] = 255

            # LAST ROW  
            if i == len(mask)-1 and (0 < j < len(mask)-1):
                if mask[i][j] == 255:
                    new[i][j-1] = 255
                    new[i][j] = 255
                    new[i][j+1] = 255
                    new[i-1][j-1] = 255
                    new[i-1][j] = 255
                    new[i-1][j+1] = 255

            # FIRST COLUMN
            if j == 0 and (0 < i < len(mask)-1):
                if mask[i][0] == 255:
                    new[i-1][j] = 255
                    new[i-1][j+1] = 255
                    new[i][j] = 255
                    new[i][j+1] = 255
                    new[i+1][j+1] = 255
                    new[i+1][j] = 255

            # LAST COLUMN
            if j == len(mask)-1 and(0 < i < len(mask)-1):
                if mask[j][i] == 255:
                    new[i-1][j] = 255
                    new[i-1][j-1] = 255
                    new[i][j-1] = 255
                    new[i][j] = 255
                    new[i+1][j-1] = 255
                    new[i+1][j] = 255

            elif (0 < i < len(mask)-1 and 0 < j < len(mask)-1) and mask[i][j] == 255:
                new[i-1][j-1] = 255
                new[i-1][j] = 255
                new[i-1][j+1] = 255
                new[i][j-1] = 255
                new[i][j] = 255
                new[i][j+1] = 255
                new[i+1][j-1] = 255
                new[i+1][j] = 255
                new[i+1][j+1] = 255
            
    new = cv2.merge((new,new,new))  # CONVERT INTO 3 CHANNEL
    new = cv2.resize(new, (src.shape[0], src.shape[1]), interpolation = cv2.INTER_AREA) 
    
    return new

def data_creation(background_path,src_path_dim, src_path, mask_path, out_save_path, mask_save_path):
  """
  Creates new data with cv2.seamless clone
  :param background_path: path of background images with the types of images 
  e.g.('/content/drive/MyDrive/background/*.png')
  :param src_path_dim: shape of source image
  :param src_path: path of source images with the type of images
  :param mask_path: path of mask images with the type of images
  :param out_save_path: path for saving final image
  :param mask_save_path: path for saving final image 
  """
  for j in range(len(glob.glob(background_path))):  
# If your images are screenshots from "Google Earth" (with buttons on screen) use the upper dst variable to crop only the image part(change the shape (1000,770) if necessary)
#   dst = cv2.resize(center_crop(cv2.imread(glob.glob(background_path)[j]),(1000,770)),src_path_dim, interpolation=cv2.INTER_AREA) 
    dst = cv2.resize(cv2.imread(glob.glob(background_path)[j]),src_path_dim, interpolation=cv2.INTER_AREA)
    for i in range(len(glob.glob(src_path))):
# Set angle variable to 0 if you dont want to rotate images
      angle = random.randint(0,360)
      src = sorted(glob.glob(src_path))[i]
      mask = sorted(glob.glob(mask_path))[i]
      new = thicker_seg_mask(src,mask)
      width, height, channels = dst.shape
      center = (int(height/2), int(width/2))
      out = cv2.seamlessClone(rotate(cv2.imread(src), angle=angle), dst, rotate(new, angle=angle), center, cv2.NORMAL_CLONE)

# Change names of saved files if needed
      cv2.imwrite(out_save_path + f'/{i}_{j}.jpg', out)
      cv2.imwrite(mask_save_path + f'/{i}_{j}.jpg', rotate(cv2.imread(mask), angle=angle))